In [ ]:
#Importing the libraries 
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
import cv2
from matplotlib import pyplot as plt
from PIL import Image
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# Loading the images and labels dataset
testImages = np.load('covid-19-dataset/testimage.npy')
testLabels = pd.read_csv('covid-19-dataset/testLabels.csv')
trainImages = np.load('covid-19-dataset/trainimage.npy')
trainLabels = pd.read_csv('covid-19-dataset/trainLabels.csv')

In [ ]:
trainImages[0][0][0]

In [ ]:
# print the sizes of array and dataframes
print("Length of testImages is",len(testImages))
print("Shape of testLabels is",testLabels.shape)
print("Length of trainImages is",len(trainImages))
print("Shape of trainImages is",trainLabels.shape)

In [ ]:
# displaying the first trainImage
plt.imshow(trainImages[0], cmap='gray')
plt.show()

In [ ]:
# displaying the first testImage
plt.imshow(testImages[0], cmap='gray')
plt.show()

In [ ]:
# Exploring the data - train data
trainLabels.value_counts()

In [ ]:
# Exploring the data - test data
testLabels.value_counts()

In [ ]:
# print unique labels in train data
trainLabels['Label'].unique()

In [ ]:
# print unique labels in test data
testLabels['Label'].unique()

In [ ]:
# Gaussian Blur on train image
cv2.imwrite('images\sampletrain.jpg', trainImages[0])

im = cv2.imread('images\sampletrain.jpg')

# load image
trainimage = Image.open('images\sampletrain.jpg')

# Gaussian Blur
Gaussian = cv2.GaussianBlur(im, (5, 5), 0)

# plot all both images using matplotlib
fig, ax = plt.subplots(1,2)

ax[0].imshow(trainimage)
ax[1].imshow(Gaussian)

ax[0].set_title("Original Image")
ax[1].set_title("Gaussian Blurring")

fig.tight_layout()
plt.show()

In [ ]:
# Gaussian Blur on test image
cv2.imwrite('images\sampletest.jpg', testImages[0])

im = cv2.imread('images\sampletest.jpg')

# load image
testimage = Image.open('images\sampletest.jpg')

# Gaussian Blur
Gaussian = cv2.GaussianBlur(im, (5, 5), 0)

# plot all both images using matplotlib
fig, ax = plt.subplots(1,2)

ax[0].imshow(testimage)
ax[1].imshow(Gaussian)

ax[0].set_title("Original Image")
ax[1].set_title("Gaussian Blurring")

fig.tight_layout()
plt.show()


In [ ]:
# Normalization of train data 
from numpy import asarray
pixels = asarray(trainimage)
# confirm pixel range is 0-255
print('Data Type: %s' % pixels.dtype)
print('Min: %.3f, Max: %.3f' % (pixels.min(), pixels.max()))
# convert from integers to floats
pixels = pixels.astype('float32')
# normalize to the range 0-1
pixels /= 255.0
# confirm the normalization
print('Min: %.3f, Max: %.3f' % (pixels.min(), pixels.max()))

In [ ]:
# global centering (subtract mean) of train data image

pixels = asarray(trainimage)
# convert from integers to floats
pixels = pixels.astype('float32')
# calculate global mean
mean = pixels.mean()
print('Mean: %.3f' % mean)
print('Min: %.3f, Max: %.3f' % (pixels.min(), pixels.max()))
# global centering of pixels
pixels = pixels - mean
# confirm it had the desired effect
mean = pixels.mean()
print('Mean: %.3f' % mean)
print('Min: %.3f, Max: %.3f' % (pixels.min(), pixels.max()))

In [ ]:
norm = np.linalg.norm(trainImages)
normal_trainImages = trainImages/norm
print(normal_trainImages)

In [ ]:
normal_trainImages.shape

In [ ]:
# flating the training data for dense layers
#trainImages_1 = trainImages.reshape((trainImages.shape[0], np.prod(trainImages.shape[1:])))
#testImages_1 = testImages.reshape((testImages.shape[0], np.prod(testImages.shape[1:])))
trainImages_1 = trainImages.reshape((trainImages.shape[0],-1))
testImages_1 = testImages.reshape((testImages.shape[0],-1))

trainLabels_x = np.array(trainLabels)
testLabels_x = np.array(testLabels)

trainLabels_1 = trainLabels_x.reshape((trainLabels_x.shape[0]))
testLabels_1 = testLabels_x.reshape((testLabels_x.shape[0]))

print(trainImages_1.shape)
print(testImages_1.shape)
print(trainLabels_1.shape)
print(testLabels_1.shape)

In [ ]:
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(trainLabels_1)
print(integer_encoded.shape)

# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded),-1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded.shape)

In [ ]:
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(testLabels_1)
print(integer_encoded.shape)

# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded),-1)
t_onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(t_onehot_encoded.shape)

In [ ]:
from tensorflow.keras import utils, datasets, layers, models
model_1 = models.Sequential()
model_1.add(layers.Dense(10, activation='softmax', input_shape=(trainImages.shape[0],trainImages.shape[1])))
model_1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_1.summary()

In [ ]:
print(trainImages_1.shape[0])
print(onehot_encoded.shape)

In [ ]:
# compile the model and train it for 5 epochs
model_1.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

train_data = tf.data.Dataset.from_tensor_slices((trainImages_1, onehot_encoded))
valid_data = tf.data.Dataset.from_tensor_slices((testImages_1, t_onehot_encoded))

history = model_1.fit(train_data, epochs=150, validation_data=valid_data)

In [ ]:
# Evaluate model
model_1.evaluate(trainImages_1, onehot_encoded), model_1.evaluate(testImages_1, t_onehot_encoded)

In [ ]:
predictions = model_1.predict_classes(testImages_1)
for i in (2,3,33,36,59):
    print('%s => %d (expected %d)' % (testImages_1[i].tolist(), predictions[i], t_onehot_encoded[i]))

In [ ]:
# val_loss
# acc
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.ylim((0,2))